In [3]:
import pandas as pd
        
pure_isw = pd.read_csv('pure_isw.csv')
pure_isw

,text_for_vect2,date
0,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-25
1,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-26
2,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-27
3,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-28
4,"[ ' frederick ' , ' kagan ' , ' george ' , ' b...",2022-03-01
...,...,...
324,"[ ' karolina ' , ' hird ' , ' grace ' , ' mapp...",2023-01-21
325,"[ ' russian ' , ' offensive ' , ' campaign ' ,...",2023-01-22
326,"[ ' russian ' , ' offensive ' , ' campaign ' ,...",2023-01-23
327,"[ ' karolina ' , ' hird ' , ' riley ' , ' bail...",2023-01-24


In [3]:
from sklearn.model_selection import train_test_split
train_docs, test_docs = train_test_split(pure_isw, test_size=5, random_state=40)

train_docs.shape, test_docs.shape

((324, 2), (5, 2))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer(max_features=1000)
doc_train_matrix = vectorizer.fit_transform(train_docs["text_for_vect2"])
words = vectorizer.get_feature_names_out()

doc_test_matrix = vectorizer.transform(test_docs["text_for_vect2"])
doc_test_matrix



<5x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1771 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda = LDA(n_components=50, learning_method='batch', evaluate_every=3, random_state=42, max_iter=10000)
lda.fit(doc_train_matrix)

LatentDirichletAllocation(evaluate_every=3, max_iter=10000, n_components=50,
                          random_state=42)

In [12]:
topic_lables = ["Theme{}".format(i) for i in range (1, 50)]

In [13]:
import joblib
joblib.dump(lda, "lda_10000_iter.pkl")
lda = joblib.load("lda_10000_iter.pkl")


In [14]:
topics_count = lda.components_
topics_count[:8]

array([[0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       ...,
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02]])

# FROM GENSIM 

In [4]:
pure_isw

,text_for_vect2,date
0,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-25
1,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-26
2,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-27
3,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-28
4,"[ ' frederick ' , ' kagan ' , ' george ' , ' b...",2022-03-01
...,...,...
324,"[ ' karolina ' , ' hird ' , ' grace ' , ' mapp...",2023-01-21
325,"[ ' russian ' , ' offensive ' , ' campaign ' ,...",2023-01-22
326,"[ ' russian ' , ' offensive ' , ' campaign ' ,...",2023-01-23
327,"[ ' karolina ' , ' hird ' , ' riley ' , ' bail...",2023-01-24


In [18]:
import gensim
from gensim.utils import simple_preprocess
tokenized_text = [simple_preprocess(text) for text in pure_isw["text_for_vect2"]]
dic = gensim.corpora.Dictionary(tokenized_text)
bow_corpus = [dic.doc2bow(doc) for doc in tokenized_text]
lda = gensim.models.LdaMulticore(bow_corpus, id2word=dic, num_topics=100, passes=15, workers=4)

In [19]:
# vectorized_text = [lda[doc] for doc in bow_corpus]
# vectorized_data = pd.DataFrame({
#     'vectorized_text': vectorized_text,
#     'date': pure_isw['date']
# })

topic_probabilities = [lda.get_document_topics(doc, minimum_probability=0) for doc in bow_corpus]

# Create DataFrame to store the results
data = []

# Iterate through each document's topic probabilities and add them to the DataFrame
for i, probs in enumerate(topic_probabilities):
    row = {'date': pure_isw['date'][i]}
    for topic, prob in probs:
        row[f'topic_{topic+1}'] = prob
    data.append(row)

# Convert list of dictionaries to DataFrame
vectorized_data = pd.DataFrame(data)


In [22]:
vectorized_data.to_csv('isw_lda.csv', index=False)

In [21]:
vectorized_data

,date,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,2022-02-25,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.999076,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
1,2022-02-26,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.000008,0.991173,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
2,2022-02-27,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,...,0.000007,0.000007,0.999330,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
3,2022-02-28,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,...,0.000007,0.000007,0.914613,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
4,2022-03-01,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.985385,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,2023-01-21,0.000004,0.000004,0.000004,0.000004,0.529134,0.000004,0.000004,0.000004,0.000004,...,0.000004,0.000004,0.007795,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004
325,2023-01-22,0.000006,0.000006,0.000006,0.000006,0.706544,0.000006,0.000006,0.000006,0.000006,...,0.000006,0.000006,0.067775,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006
326,2023-01-23,0.000004,0.000004,0.000004,0.000004,0.605457,0.003095,0.000004,0.000004,0.000004,...,0.000004,0.000004,0.006186,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004
327,2023-01-24,0.000003,0.000003,0.000003,0.000003,0.269717,0.000003,0.000003,0.000003,0.000003,...,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
